In [1]:
import dask.dataframe as dd
import pandas as pd, numpy as np
from pybedtools import BedTool

In [5]:
# df=dd.read_table('GPL13534-11288.txt',dtype={'Coordinate_36':'object','Genome_Build': 'float64',
#        'MAPINFO': 'float64',
#        'RANGE_END': 'int',
#        'RANGE_START': 'int',
#        'SPOT_ID': 'object'})
df=pd.read_csv('GPL13534-11288.txt',sep='\t', )


In [3]:
df.columns

Index(['ID', 'Name', 'AddressA_ID', 'AlleleA_ProbeSeq', 'AddressB_ID',
       'AlleleB_ProbeSeq', 'Infinium_Design_Type', 'Next_Base',
       'Color_Channel', 'Forward_Sequence', 'Genome_Build', 'CHR', 'MAPINFO',
       'SourceSeq', 'Chromosome_36', 'Coordinate_36', 'Strand', 'Probe_SNPs',
       'Probe_SNPs_10', 'Random_Loci', 'Methyl27_Loci', 'UCSC_RefGene_Name',
       'UCSC_RefGene_Accession', 'UCSC_RefGene_Group', 'UCSC_CpG_Islands_Name',
       'Relation_to_UCSC_CpG_Island', 'Phantom', 'DMR', 'Enhancer',
       'HMM_Island', 'Regulatory_Feature_Name', 'Regulatory_Feature_Group',
       'DHS', 'RANGE_START', 'RANGE_END', 'RANGE_GB', 'SPOT_ID'],
      dtype='object')

In [6]:
{'UCSC_RefGene_Accession': 17,
'UCSC_RefGene_Group': 96135,
'UCSC_CpG_Islands_Name': 11,
'Relation_to_UCSC_CpG_Island': 49197,
'Phantom': 2,
'DMR': 12231,
'Enhancer': 102559,
'HMM_Island': 3,
'Regulatory_Feature_Name': 1,
'Regulatory_Feature_Group': 4045,
'DHS': 59916}.keys()

dict_keys(['UCSC_RefGene_Accession', 'UCSC_RefGene_Group', 'UCSC_CpG_Islands_Name', 'Relation_to_UCSC_CpG_Island', 'Phantom', 'DMR', 'Enhancer', 'HMM_Island', 'Regulatory_Feature_Name', 'Regulatory_Feature_Group', 'DHS'])

In [4]:
df=df.set_index('ID')

In [82]:
from collections import Counter

In [83]:
from functools import reduce

In [84]:
k='Relation_to_UCSC_CpG_Island'
Counter(';'.join(df[k][(~df[k].isnull())].astype(str).tolist()).split(';')).most_common(12)

[('Island', 150254),
 ('N_Shore', 62870),
 ('S_Shore', 49197),
 ('N_Shelf', 24844),
 ('S_Shelf', 22300)]

In [98]:
capsules={}
for k in df.columns.values[21:-4]:
    capsules[k]={k2:[] for k2 in np.unique(np.array(';'.join(df[k][(~df[k].isnull())].astype(str).tolist()).split(';')))}
    
    

In [99]:
for cpg in df.index:
    for k in df.columns.values[21:-4]:
        v=str(df.loc[cpg,k])
        for v2 in v.split(';'):
            if v2!='nan':
                capsules[k][v2].append(cpg)
        
            

In [107]:
import pickle

In [116]:
{k:int(np.mean(([len(capsules[k][k2]) for k2 in capsules[k]]))) for k in  list(capsules.keys())}

{'UCSC_RefGene_Accession': 21,
 'UCSC_RefGene_Group': 114522,
 'UCSC_CpG_Islands_Name': 11,
 'Relation_to_UCSC_CpG_Island': 61893,
 'Phantom': 2,
 'DMR': 12445,
 'Enhancer': 102559,
 'HMM_Island': 3,
 'Regulatory_Feature_Name': 3,
 'Regulatory_Feature_Group': 21261,
 'DHS': 59916}

In [108]:
pickle.dump(capsules,open('capsules.p','wb'))#['Enhancer']#.keys()#['False']=np.setdiff1d(df.index,capsules['Enhancer']['True'])

In [13]:
def to_int(chrom):
    try:
        chrom=int(chrom)
    except:
        pass
    return chrom

In [70]:
df=df[['Chromosome_36','Coordinate_36','Name']].dropna()
df=df[df['Chromosome_36']!='MULTI']
df.loc[:,'Chromosome_36']=np.vectorize(to_int)(df.loc[:,'Chromosome_36'])
#df['Start']=df['Coordinate_36'].astype(int)



In [71]:
df['Chromosome_36'].unique()

array(['Y', 'X', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'],
      dtype=object)

In [74]:
df['Start']=df['Coordinate_36'].astype(int)
df['End']=df['Start']+1

In [8]:
df=df[['CHR','RANGE_START','RANGE_END','Name']]
df=df[~df['RANGE_START'].isnull()]
df.loc[:,['RANGE_START','RANGE_END']]=df.loc[:,['RANGE_START','RANGE_END']].astype(int)

In [14]:
df.loc[:,'CHR']=np.vectorize(to_int)(df.loc[:,'CHR'])#.unique()

In [15]:
df['CHR'].unique()

array(['Y', 'X', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'],
      dtype=object)

In [16]:
annotation450k=BedTool.from_dataframe(df[['CHR','RANGE_START','RANGE_END','Name']]) # .compute()

In [17]:
annotation450k.head()

Y	8553009	8553132	cg00035864
 Y	9363356	9363479	cg00050873
 Y	25314171	25314294	cg00061679
 Y	22741795	22741918	cg00063477
 Y	21664296	21664419	cg00121626
 Y	21239348	21239471	cg00212031
 Y	8148233	8148356	cg00213748
 Y	15815688	15815811	cg00214611
 Y	4868996	4869119	cg00223952
 Y	6133740	6133863	cg00243321
 

In [75]:
annotation450k=BedTool.from_dataframe(df[['Chromosome_36','Start','End','Name']]) # .compute()

In [18]:
annotation450k.saveas('450kannotations.bed')

<BedTool(450kannotations.bed)>

In [ ]:
df.loc[:,['Coordinate_36','RANGE_END']]=df.loc[:,['Coordinate_36','RANGE_END']].astype(int)
df['End']=df['Start']+1
df=df[['CHR','Start','End','Name']]
df=df[df['Chromosome_36']!='MULTI']
df.loc[:,'Chromosome_36']=np.vectorize(to_int)(df.loc[:,'Chromosome_36'])